## Постановка задачи
Загрузим подготовленные данные из HDF5. Разделим данные на обучающие и проверочные и построим двухслойный перцептрон для типа облака Fish.

Проведем оценку качества предсказания по коэффициенту сходства.

Данные:
* https://video.ittensive.com/machine-learning/clouds/clouds.data.h5 (959 Мб)

Соревнование: https://www.kaggle.com/c/understanding_cloud_organization/

© ITtensive, 2020

### Подключение библиотек

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

### Используемые функции

In [2]:
image_x = 525
image_y = 350
def mask_rate (a, x, y):
    b = a//1400 + 0.0
    return np.round(x*(b*x//2100) + y*(a%1400)//1400).astype("uint32")

def calc_mask (px, x=image_x, y=image_y):
    p = np.array([int(n) for n in px.split(' ')]).reshape(-1,2)
    mask = np.zeros(x*y, dtype='uint8')
    for i, l in p:
        mask[mask_rate(i, x, y) - 1:mask_rate(l+i, x, y)] = 1
    return mask.reshape(y,x).transpose()

def calc_dice (x):
    dice = 0
    px = x["EncodedPixels"] 
    if px != px and x["target"] == 0:
        dice = 1
    elif px == px and x["target"] == 1:
        mask = calc_mask(px).flatten()
        target = np.ones(image_x*image_y, dtype='uint8')
        dice = 2*np.sum(target[mask==1])/(np.sum(target)+np.sum(mask))
    return dice

### Загрузка данных

In [3]:
clouds = pd.read_hdf('clouds.data.h5')
print (clouds.head())

    0   1    2    3    4    5    6    7    8    9  ...  183745  183746  \
0   0   0    0    0    0    0    0    0    0    0  ...     102     113   
1  71  42   31   38   48   68   70   46   24   21  ...      38      57   
2  97  98  100  102  105  107  109  110  111  112  ...       0       0   
3  87  86   89   93   93   88   86   86   81   81  ...     165     164   
4  18  19   21   21   20   19   17   16   14   15  ...     125     137   

   183747  183748  183749        Image  \
0     102      96     116  0011165.jpg   
1      82     110     133  002be4f.jpg   
2       0       0       0  0031ae9.jpg   
3     108      70      72  0035239.jpg   
4      92      59      48  003994e.jpg   

                                                Fish  \
0  264918 937 266318 937 267718 937 269118 937 27...   
1  233813 878 235213 878 236613 878 238010 881 23...   
2  3510 690 4910 690 6310 690 7710 690 9110 690 1...   
3                                                NaN   
4  2367966 18 2367985 

Оставим только данные по Fish, на них обучим модель

In [4]:
clouds.drop(labels=["Image", "Flower", "Gravel", "Sugar"],
    axis=1, inplace=True)

### Разделение данных
Разделим всю выборку на 2 части случайным образом: 80% - для обучения модели, 20% - для проверки точности модели.

In [5]:
clouds_train, clouds_test = train_test_split(clouds, test_size=0.2)
clouds_train = pd.DataFrame(clouds_train)
clouds_test = pd.DataFrame(clouds_test)
del clouds
print (clouds_train.head())

        0    1    2    3    4    5    6    7    8    9  ...  183741  183742  \
1841   42   32   32   25    4    0    2    2    0    0  ...      19      18   
5196  149  149  148  147  147  146  146  145  146  145  ...      90      92   
2006  104  134  114   56   30   34   39   43   50   77  ...      46      49   
1948   39   36   32   30   31   32   33   33   32   32  ...     220     227   
4561  161  159  145  163  180  165  153  143  142  142  ...     161     214   

      183743  183744  183745  183746  183747  183748  183749  \
1841      17      17      17      17      17      17      18   
5196      95      92      98     118     119     102      97   
2006      43      43      44      45      45      46      47   
1948     225     227     220     218     211     212     226   
4561     236     241     228     235     219     197     202   

                                                   Fish  
1841  1768642 682 1770042 682 1771442 682 1772842 68...  
5196                    

### Двухслойный перцептрон
![](https://www.researchgate.net/profile/Yan-Fu_Li/publication/281534869/figure/fig4/AS:668235573755913@1536331184829/Architecture-of-a-MLP-NN-model.png)
Последовательно рассчитываем коэффициенты для пакетов по 100 изображений, иначе может не поместиться в оперативную память.

Используем warm_start=True, чтобы переиспользовать предыдущие параметры.

In [7]:
y = clouds_train["Fish"].notnull().astype("int8")
x = pd.DataFrame(clouds_train).drop(labels=["Fish"], axis=1)
model = MLPClassifier(hidden_layer_sizes=(31,),
                     max_iter=20, activation="logistic",
                     verbose=10, random_state=1, learning_rate_init=.02,
                     warm_start=True)

In [12]:
for i in range(len(clouds_train)//100):
    model.partial_fit(x[i:i+100], y[i:i+100], classes=[0, 1])

Iteration 1, loss = 0.72517492
Iteration 2, loss = 0.69139934
Iteration 3, loss = 0.69218115
Iteration 4, loss = 0.68995657
Iteration 5, loss = 0.69197143
Iteration 6, loss = 0.69507116
Iteration 7, loss = 0.69910045
Iteration 8, loss = 0.69878797
Iteration 9, loss = 0.69840872
Iteration 10, loss = 0.70072479
Iteration 11, loss = 0.70232555
Iteration 12, loss = 0.70359634
Iteration 13, loss = 0.70379597
Iteration 14, loss = 0.70544680
Iteration 15, loss = 0.70782095
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 16, loss = 0.70839137
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 17, loss = 0.70856360
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 18, loss = 0.70944267
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Iteration 19, loss = 0.70964819
Training loss did not improve more than t

In [13]:
del x
del y

### Предсказание значений

In [14]:
result = pd.DataFrame({"EncodedPixels": clouds_test["Fish"]})
result["target"] = model.predict(clouds_test.drop(labels=["Fish"],
                                                 axis=1))
print (result.head(10))

                                          EncodedPixels  target
2545                                                NaN       0
588   1663220 690 1664620 690 1666020 690 1667420 69...       0
1852                                                NaN       0
5237  1332363 411 1333763 411 1335163 411 1336563 41...       0
927   9477 314 10877 314 12277 314 13677 314 15077 3...       0
5096  6604 394 8004 394 9404 394 10804 394 12204 394...       0
4846  2073 400 3473 393 3870 3 4873 390 5264 9 6273 ...       0
3162  280828 366 282228 366 283628 366 285028 366 28...       1
1840  373209 10 373223 4 373230 3 373234 2 373237 1 ...       0
3340  579631 833 581031 833 582431 833 583831 833 58...       0


### Оценка по Дайсу
Пока будем считать, что при определении типа облака на изображении, оно целиком размещено на фотографии: т.е. область облака - это все изображение.

Нет облаков - 0.5, опорные векторы - 0.3

In [15]:
dice = result.apply(calc_dice, axis=1, result_type="expand")
print ("MLP, Fish:", round(dice.mean(), 3))

MLP, Fish: 0.498
